In [24]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "8"

In [25]:
from tqdm.auto import tqdm
from typing import List, Tuple
import re
DEVICE = 'cuda:0'
import torch

In [26]:
import sys
sys.path.append('../../..')
print(os.path.realpath("."))

/mount/arbeitsdaten41/projekte/asr-2/vaethdk/cts_newcodebase_rollback/conversational-tree-search/generation/onboarding/llama


In [27]:
a = torch.zeros(10,10,device=DEVICE)

In [28]:
# !GITHUB_ACTIONS=true pip install auto-gptq

In [29]:
from data.dataset import OnboardingGraphDataset, DataAugmentationLevel, NodeType, DialogNode, Question
human_data_train = OnboardingGraphDataset('en/onboarding/train_graph.json', 'en/onboading/train_graph.json', False, augmentation=DataAugmentationLevel.NONE, resource_dir="../../../resources")

- not using synonyms
===== Dataset Statistics =====
- files:  en/onboarding/train_graph.json en/onboading/train_graph.json
- synonyms: False
- depth: 13  - degree: 9
- answers: 43
- questions: 0
- loaded original data: True
- loaded generated data: False


In [30]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

In [31]:
model_name_or_path = "TheBloke/upstage-llama-30b-instruct-2048-GPTQ"
model_basename = "gptq_model-4bit--1g"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          use_fast=True,
                                          cache_dir="/mount/arbeitsdaten/asr-2/vaethdk/resources/weights/")


In [32]:
model = AutoGPTQForCausalLM.from_quantized("/mount/arbeitsdaten/asr-2/vaethdk/resources/weights/TheBloke--upstage-llama-30b-instruct-2048-GPTQ",
        # model_basename=model_basename,
        # revision="gptq-4bit-32g-actorder_True",
        use_safetensors=True,
        trust_remote_code=False,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

KeyboardInterrupt: 

### System:
{System}

### User:
{User}

### Assistant:
{Assistant}

In [ ]:
system = """You are a helpful assistant creating a list of FAQ-style questions from given facts.
Only generate questions that can be answered by the given facts, without any external knowledge.
Remove some information, especially nouns and named entities, between generated questions.
Use casual language.
Order the generated paraphrases in a numbered list."""
user = 'Generate 10 FAQ-style questions from the fact: "In the US, you are entitled to 30$ per day, minus any free meals which you choose to decline."'


prompt = f"""
### System:
{system}

### User:
{user}

### Assistant:"""

set_seed(42)


In [ ]:
# input_ids = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
# print(tokenizer.decode(output[0]))

KeyboardInterrupt: 

In [ ]:
# # try batch processing
# tokenizer.pad_token = tokenizer.eos_token
# input_ids = tokenizer([prompt, prompt + prompt], return_tensors='pt', padding=True, ).input_ids.cuda()
# output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)

## Generate Question Synonyms

In [ ]:
def generate_prompt(system: str, user: str) -> str:
    return f"""
    ### System:
    {system}

    ### User:
    {user}

    ### Assistant:"""

def generate_output(prompt: str, temperature: float = 0.7, max_new_tokens: int = 512) -> torch.FloatTensor:
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=temperature, max_new_tokens=max_new_tokens)
    return tokenizer.decode(output[0])

In [ ]:
human_data_train = OnboardingGraphDataset('en/onboarding/train_graph.json', 'en/onboading/train_graph.json', False, augmentation=DataAugmentationLevel.NONE, resource_dir="../../../resources")

- not using synonyms
===== Dataset Statistics =====
- files:  en/onboarding/train_graph.json en/onboading/train_graph.json
- synonyms: False
- depth: 13  - degree: 9
- answers: 43
- questions: 0
- loaded original data: True
- loaded generated data: False


In [ ]:
# check that we don't have any answer synonyms
for node in human_data_train.node_list:
    for question in node.questions:
        human_data_train.question_list.remove(question)
        del human_data_train.questions_by_key[question.key]
    node.questions.clear()
assert len(human_data_train.question_list) == 0
assert len(human_data_train.questions_by_key) == 0

In [ ]:
def parse_output(original_question: str, prompt: str, output: str, num_paraphrases: int) -> List[str]:
    # remove prompt from output first (ends at ### ASSISTANT: )
    questions = []
    cleaned = output[len(prompt):]
    
    if not "1." in cleaned: 
        print("NO LIST FOR QUESTION", original_question)
        return questions
    
    for i in range(1, num_paraphrases+1):
        if not f"{i}." in cleaned: 
            print(f" - NO {i}. CANDIDATE FOR QUESTION", original_question)
            continue

        start_idx = cleaned.find(f"{i}.") # find i. line
        end_idx = cleaned.find("\n", start_idx) # read until line end 
        if i == num_paraphrases and end_idx == -1:
            # last line might not have line break
            end_idx = len(cleaned)
        if start_idx == -1 or end_idx == -1:
            print(f" - INDEX PROBLEM FOR {i}. CANDIDATE: ({start_idx}, {end_idx})")
            continue
        # parse answer
        questions.append(cleaned[start_idx:end_idx].replace("</s>", "").strip())

        cleaned = cleaned[end_idx:] # remove i. line
    return questions

# V1

In [ ]:
from data.dataset import NodeType, Question
import time

set_seed(42)

system = """You are a helpful assistant creating a list of FAQ-style questions from given facts.
Only generate questions that can be answered by the given facts, without any external knowledge.
Remove some information, especially nouns and named entities, between generated questions.
Use casual language.
Order the generated paraphrases in a numbered list."""

def user(answer_text: str, num_paraphrases: int) -> str:
    return f'Generate {num_paraphrases} FAQ-style questions from the fact: "{answer_text}"'

NUM_QUESTIONS = 10
TEMPERATURE = 0.7
MAX_NEW_TOKENS = 1024
generated_data = {}

for node in tqdm(human_data_train.nodes_by_type[NodeType.INFO]):
    prompt = generate_prompt(system=system, user=user(node.text, NUM_QUESTIONS))
    gen = generate_output(prompt=prompt, temperature=TEMPERATURE, max_new_tokens=MAX_NEW_TOKENS)
    candidates = parse_output(original_question=node.text, prompt=prompt, output=gen, num_paraphrases=NUM_QUESTIONS)
    for candidate in candidates:
        key = str(time.time()).replace(".", "")
        generated_data[key] = {
            "dialog_node_key": node.key,
            "key": key,
            "text": candidate,
        }

 11%|█▏        | 7/61 [51:33<6:37:45, 441.95s/it] 


KeyboardInterrupt: 

In [ ]:
import json

cleaned_data = {}
for key in generated_data:
    node = human_data_train.nodes_by_key[generated_data[key]['dialog_node_key']]
    cleaned_data[key] = generated_data[key]
    for i in range (1, NUM_QUESTIONS+1):
        cleaned_data[key]['text'] = cleaned_data[key]['text'].replace(f"{i}.", "").strip()
    cleaned_data[key]["node_text"] = node.text
    cleaned_data[key]["node_type"] = node.node_type.value

with open("resources/en/onboarding/generated/train_questions_v1.json", "w") as f:
    json.dump(cleaned_data, f)

# DATA GENERATION V2: SHORTER QUESTIONS


In [ ]:
from data.dataset import NodeType, Question
import time

set_seed(42)

system = """You are a helpful assistant creating a list of FAQ-style questions from given facts.
Only generate questions that can be answered by the given facts, without any external knowledge.
Remove some information, especially nouns and named entities, between generated questions.
Use casual language.
Prefer short questions.
Order the generated questions in a numbered list."""

def user(answer_text: str, num_paraphrases: int) -> str:
    return f'Generate {num_paraphrases} short FAQ-style questions from the fact: "{answer_text}"'

NUM_QUESTIONS = 10
TEMPERATURE = 0.7
MAX_NEW_TOKENS = 1024
generated_data = {}

for node in tqdm(human_data_train.nodes_by_type[NodeType.INFO]):
    prompt = generate_prompt(system=system, user=user(node.text, NUM_QUESTIONS))
    gen = generate_output(prompt=prompt, temperature=TEMPERATURE, max_new_tokens=MAX_NEW_TOKENS)
    candidates = parse_output(original_question=node.text, prompt=prompt, output=gen, num_paraphrases=NUM_QUESTIONS)
    for candidate in candidates:
        key = str(time.time()).replace(".", "")
        generated_data[key] = {
            "dialog_node_key": node.key,
            "key": key,
            "text": candidate,
        }

100%|██████████| 80/80 [11:34:35<00:00, 520.95s/it]  


In [ ]:
import json

cleaned_data = {}
for key in generated_data:
    node = human_data_train.nodes_by_key[generated_data[key]['dialog_node_key']]
    cleaned_data[key] = generated_data[key]
    for i in range (1, NUM_QUESTIONS+1):
        cleaned_data[key]['text'] = cleaned_data[key]['text'].replace(f"{i}.", "").strip()
    cleaned_data[key]["node_text"] = node.text
    cleaned_data[key]["node_type"] = node.node_type.value

with open("resources/en/onboarding/generated/train_questions_v2.json", "w") as f:
    json.dump(cleaned_data, f)

# DATA GENERATION V3: SHORTER QUESTIONS + SPLIT NODE CONTEXT

1. Try to generate shorter questions (-> change prompt)
2. Try to generate more diverse questions
    1. Detect relevant sentences in node text via NER tool (also detects time, quantities, ...)
    2. Generate questions for whole node context, then for only relevant sentences / sub-sentences of node
    3. Choose amount of questions to be generated depending on amount of extracted NERs?

In [ ]:
# !pip install stanza

     |████████████████████████████████| 802 kB 14.3 MB/s eta 0:00:01
     |████████████████████████████████| 361 kB 119.2 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=d6fa52fd4eea49ae84298a10e2c77bf642c90e75abcdd4db626c8b0309a7e8f2
  Stored in directory: /home/users2/vaethdk/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji
You should consider upgrading via the '/fs/scratch/users/vaethdk/adviser_reisekosten/.env/bin/python -m pip install --upgrade pip' command.


In [ ]:
import stanza

In [ ]:
torch.cuda.device_count()

1

In [ ]:
# stanza.download(lang="en", model_dir=".models/")

2023-08-04 16:21:45 INFO: Downloading default packages for language: en (English) ...
2023-08-04 16:21:56 INFO: Finished downloading models and saved to .models/.


In [ ]:
nlp = stanza.Pipeline('en', processors='tokenize,ner', device="cuda:0")

2023-08-11 11:18:46 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-08-11 11:18:49 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2023-08-11 11:18:49 INFO: Using device: cuda:0
2023-08-11 11:18:49 INFO: Loading: tokenize
2023-08-11 11:18:52 INFO: Loading: ner
2023-08-11 11:18:52 INFO: Done loading processors!


In [ ]:
print(doc.ents)

[{
  "text": "Barack Obama",
  "type": "PERSON",
  "start_char": 0,
  "end_char": 12
}, {
  "text": "Hawaii",
  "type": "GPE",
  "start_char": 25,
  "end_char": 31
}]


In [ ]:
from statistics import mean

nodes_with_ner = 0
nodes_without_ner = 0
avg_node_ner = []

for node in tqdm(human_data_train.nodes_by_type[NodeType.INFO]):
    context = nlp(node.text)
    if len(context.ents) > 0:
        nodes_with_ner += 1
        avg_node_ner.append(len(context.ents))
    else:
        nodes_without_ner += 1

print("TOTAL INFO NODES", len(human_data_train.nodes_by_type[NodeType.INFO]))
print("NODES WITH NER", nodes_with_ner)
print("NODES WITHOUT NER", nodes_without_ner)
print("AVG NER PER NODE WITH NER", mean(avg_node_ner))

100%|██████████| 80/80 [00:04<00:00, 16.02it/s]

TOTAL INFO NODES 80
NODES WITH NER 33
NODES WITHOUT NER 47
AVG NER PER NODE WITH NER 2.090909090909091


In [ ]:
avg_node_sentence_length = []

for node in human_data_train.nodes_by_type[NodeType.INFO]:
    avg_node_sentence_length.append(node.text.count("."))

print("MAX #SENTENCES PER NODE", max(avg_node_sentence_length))
print("AVG #SENTENCES PER NODE", mean(avg_node_sentence_length))

MAX #SENTENCES PER NODE 6
AVG #SENTENCES PER NODE 1.8


In [ ]:
def extract_ner_sentences(node: DialogNode) -> List[Tuple[str, str]]:
    """
    Extract all sentences from node text that mention NER's.
    Returns them as a list of tuples, where each tuple contains
        1. the name of the entity
        2. the sentence containing that entity
    """
    results = []
    context = nlp(node.text)
    entities = context.ents
    for entity in entities:
        start_idx = entity.start_char
        end_idx = entity.end_char
        # expand start index to beginning of sentence
        while start_idx > 0 and node.text[start_idx-1] != ".":
            start_idx -= 1
        # expand end index to end of sentence
        while end_idx < len(node.text) and node.text[end_idx-1] != ".":
            end_idx += 1
        results.append((entity.text, node.text[start_idx:end_idx]))
    return results

In [ ]:
# find a testing candidate
for node in human_data_train.nodes_by_type[NodeType.INFO]:
    results = extract_ner_sentences(node)
    if len(results) > 1:
        print(results)
        break

[('COVID-19', 'Please check the current COVID-19 travel warnings travel restrictions from the foreign ministry and the RKI.'), ('RKI', 'Please check the current COVID-19 travel warnings travel restrictions from the foreign ministry and the RKI.'), ('Department 4 (Administrative Department', ' In In extreme cases, authorization can be given by the leadership of Department 4 (Administrative Department).')]


In [ ]:
print(node.text)

Please check the current COVID-19 travel warnings travel restrictions from the foreign ministry and the RKI. Business trips to high risk areas or virus variation areas are not generally not allowed. In In extreme cases, authorization can be given by the leadership of Department 4 (Administrative Department).


In [ ]:
from data.dataset import NodeType, Question
import time

system = """You are a helpful assistant creating a list of diverse FAQ-style questions from given facts.
Only generate questions that can be answered by the given facts, without any external knowledge.
Use casual language.
Prefer short questions.
Order the generated paraphrases in a numbered list."""

def user(answer_text: str, num_paraphrases: int) -> str:
    return f'Generate {num_paraphrases} short and diverse FAQ-style questions from the fact: "{answer_text}"'


NUM_QUESTIONS = 3
TEMPERATURE = 0.7
MAX_NEW_TOKENS = 1024
generated_data = {}


for entity, sentence in tqdm(extract_ner_sentences(node)):
    prompt = generate_prompt(system=system, user=user(sentence, NUM_QUESTIONS))
    gen = generate_output(prompt=prompt, temperature=TEMPERATURE, max_new_tokens=MAX_NEW_TOKENS)
    candidates = parse_output(original_question=node.text, prompt=prompt, output=gen, num_paraphrases=NUM_QUESTIONS)
    generated_data[entity] = candidates


100%|██████████| 3/3 [03:52<00:00, 77.40s/it]


In [ ]:
import pprint
pprint.pprint(generated_data)

{'COVID-19': ['1. What should I do to stay updated on COVID-19 travel warnings '
              'and restrictions?',
              '2. Where can I find the latest information on travel advisories '
              'related to COVID-19?',
              '3. Which organizations should I refer to for understanding the '
              'current travel restrictions due to COVID-19?'],
 'Department 4 (Administrative Department': ['1. Who can grant authorization '
                                             'in extreme cases?',
                                             "2. What department's leadership "
                                             'can authorize in extreme '
                                             'situations?',
                                             "3. Which department's leadership "
                                             'has the power to authorize in '
                                             'extreme cases?'],
 'RKI': ['1. What should I do to stay upd

In [ ]:
from data.dataset import NodeType, Question
import time

system = """You are a helpful assistant creating a list of diverse FAQ-style questions from given facts.
Only generate questions that can be answered by the given facts, without any external knowledge.
Use casual language.
Prefer short questions.
Order the generated paraphrases in a numbered list."""

def user(answer_text: str, num_paraphrases: int) -> str:
    return f'Generate {num_paraphrases} short and diverse FAQ-style questions from the fact: "{answer_text}"'


NUM_QUESTIONS = 3
TEMPERATURE = 0.7
MAX_NEW_TOKENS = 1024

prompt = generate_prompt(system=system, user=user(node.text, NUM_QUESTIONS))
gen = generate_output(prompt=prompt, temperature=TEMPERATURE, max_new_tokens=MAX_NEW_TOKENS)
candidates = parse_output(original_question=node.text, prompt=prompt, output=gen, num_paraphrases=NUM_QUESTIONS)
pprint.pprint(candidates)

KeyboardInterrupt: 

In [ ]:
system = """You are a helpful assistant creating a list of diverse FAQ-style questions from given facts.
Only generate questions that can be answered by the given facts, without any external knowledge.
Use casual language.
Prefer short questions.
Order the generated paraphrases in a numbered list."""

def user(answer_text: str, num_paraphrases: int) -> str:
    return f'Generate {num_paraphrases} short and diverse FAQ-style questions from the fact: "{answer_text}"'

def user_ner(answer_text: str, ner: str, num_paraphrases: int) -> str:
    return f'Generate {num_paraphrases} short and diverse FAQ-style questions about the entity "{ner}" from the fact: "{answer_text}"'


NUM_QUESTIONS = 10
NUM_QUESTIONS_PER_SENTENCE = 3
TEMPERATURE = 0.7
MAX_NEW_TOKENS = 1024
generated_data = {}

set_seed(42)

for node in tqdm(human_data_train.nodes_by_type[NodeType.INFO]):
    # use dict indexed by generated text to filter out duplicates
    all_generations = {}
    
    # extract NERs
    named_entities = extract_ner_sentences(node)

    # Generate questions with NER sentences only, make asking about NER a requirement
    for entity, sentence in named_entities:
        prompt = generate_prompt(system=system, user=user_ner(node.text, entity, NUM_QUESTIONS_PER_SENTENCE))
        gen = generate_output(prompt=prompt, temperature=TEMPERATURE, max_new_tokens=MAX_NEW_TOKENS)
        candidates = parse_output(original_question=node.text, prompt=prompt, output=gen, num_paraphrases=NUM_QUESTIONS_PER_SENTENCE)
        for candidate_idx, candidate in enumerate(candidates):
            key = str(time.time()).replace(".", "")
            cleaned_candidate = candidate.replace(f"{candidate_idx+1}.", "").strip()
            all_generations[cleaned_candidate] = {
                "context": "ner",
                "entity": entity,
                "dialog_node_key": node.key,
                "key": key,
                "text": cleaned_candidate
            }

    # Generate questions with whole context
    num_node_level_questions = max(NUM_QUESTIONS_PER_SENTENCE, NUM_QUESTIONS - len(named_entities) * NUM_QUESTIONS_PER_SENTENCE)
    prompt = generate_prompt(system=system, user=user(node.text, num_node_level_questions))
    gen = generate_output(prompt=prompt, temperature=TEMPERATURE, max_new_tokens=MAX_NEW_TOKENS)
    candidates = parse_output(original_question=node.text, prompt=prompt, output=gen, num_paraphrases=NUM_QUESTIONS_PER_SENTENCE)
    for candidate_idx, candidate in enumerate(candidates):
        key = str(time.time()).replace(".", "")
        cleaned_candidate = candidate.replace(f"{candidate_idx+1}.", "").strip()
        all_generations[cleaned_candidate] = {
            "context": "node",
            "dialog_node_key": node.key,
            "key": key,
            "text": cleaned_candidate
        }
    
    # add filtered questions to generated dataset
    for text in all_generations:
        entry = all_generations[text]
        generated_data[entry["key"]] = entry

100%|██████████| 80/80 [12:37:21<00:00, 568.01s/it]  


In [ ]:
import json

cleaned_data = {}
for key in generated_data:
    node = human_data_train.nodes_by_key[generated_data[key]['dialog_node_key']]
    cleaned_data[key] = generated_data[key]
    for i in range (1, NUM_QUESTIONS+1):
        cleaned_data[key]['text'] = cleaned_data[key]['text'].replace(f"{i}.", "").strip()
    cleaned_data[key]["node_text"] = node.text
    cleaned_data[key]["node_type"] = node.node_type.value

with open("resources/en/onboarding/generated/train_questions_v3.json", "w") as f:
    json.dump(cleaned_data, f)